In [41]:
# Importiere rdflib
from rdflib import Graph, Literal, URIRef, BNode 
# Importiere die für diese Arbeit wichtigen Abkürzungen aus RDFLIB
from rdflib.namespace import OWL, RDF, RDFS
#Importiere die Ontologieabfragemethode aus RDFLIB
from rdflib.plugins.sparql import prepareQuery
#Importiere OWLReady2
import owlready2 as OWLREADY

# Save the link towards the ontology file containing the recipes
ontology ="RecipeInstrucWithUnderscoreAndVerbActions.rdf" 

#Define the graph
graph = Graph()
#Load the ontology
graph.parse(ontology)

#Rezeptliste
recipes = []
#Suche alle Rezepte heraus
recipeQuery = prepareQuery(
    """
    SELECT *
    WHERE{?s <http://purl.org/ProductKG/RecipeOn#id> ?o}
    """,
)
#Speichere alle Rezepte per Suchresultat in die Rezeptliste
for row in graph.query(recipeQuery):
    recipes.append(row.s)
    
print(len(recipes))

#Rezept- und Rezeptinstruktionsliste
recipePlusInstructions = []
#Für alle Rezepte---
for recipe in recipes:
    #Suche die Instruktion für das jeweilige Rezept aus. Dies wird als Objekt ?o ausgegeben
    instructionQuery = prepareQuery("SELECT * \
    WHERE{ <"+ str(recipe) +"> <http://purl.org/ProductKG/RecipeOn#instructions> ?o}")
    # Für jede Reihe aus der von der Ontologie stammenden Resultate
    for instructions in graph.query(instructionQuery):
        # Nehme das Rezept, zwinge sie zu einer Zeichenkette und addiere ein Punkt und ein Zeilenumbruch hinzu und addiere dies in die Instruktionliste
        recipePlusInstructions.append(str(recipe)+ ".\n")
        # Hole die Instrukion aus der Suche für das Rezept und addiere sie in die Instruktionliste
        recipePlusInstructions.append(instructions.o)
        # Addiere einen Zeilenumbruch
        recipePlusInstructions.append("\n")
        
#Kreiere eine Textdatei mit Schreibzugriff
file = open("input.txt","w")
#Für alle Elemente der Rezept- und Rezeptinstruktionsmenge
for row in recipePlusInstructions:
    #Schreibe sie in die Datei
    file.write(row)
#Beende den Schreibzugriff und speichere die Textdatei
file.close()

9882


In [2]:
# Setze die URI für die neue Relation
includesTask = URIRef("http://www.ease-crc.org/ont/SOMA.owl#includes_task")

# Öffne die gefilterten Rezept-Datei als Read-only
finalFile = open("output.txt","r")

# Für alle Zeilen in der Datei...
for row in finalFile:
    # Falls man in der Zeile eine Rezept-IRI findet...
    if (row.find("http://purl.org") > -1):
        #Setze eine Variabel gleich dem IRI ohne Satzumbruch
        recipeURI=row.replace("\n","")
        #Solange aus vorherigen Programmen mitgebrachte Sondersymbole eingebracht werden, entferne sie bis sie nicht mehr am Ende des Rezepts gefunden werden kann.
        while(recipeURI[len(recipeURI)-1] == "." or recipeURI[len(recipeURI)-1] == "'" or recipeURI[len(recipeURI)-1] == " "):
            recipeURI=recipeURI[:len(recipeURI)-1]
        #SONDERFALLREZEPTE!!!!
        # Die, die in ihren IRIs ein tatsächlichen Endpunkt haben
        if "Ras_Malai" in row:
            recipeURI = recipeURI + "."
        if "Power_Pumpkin_Pie_Pumpkin_pie" in row:
            recipeURI = recipeURI + "."
        if "Italian_Beef_Sandwiches" in row:
            recipeURI = recipeURI + "."
        if "fun_eggs._Huevos_div" in row:
            recipeURI = recipeURI + "."
        if "Flounder_and/or" in row:
            recipeURI = recipeURI + "."
        if "Easy_Simple_Marinated_Grilled_Chicken" in row:
            recipeURI = recipeURI + "."
        if "Savory_Herb_Roast" in row:
            recipeURI = recipeURI + "."
        # Nach Vornamen benannte Rezepte mit 's
        if "Christmas_Bombe" in row:
            recipeURI.replace("\'","'")
            recipeURI = recipeURI + "'"
        if "Smoked_Salmon_and_Udon_Noodle" in row:
            recipeURI = recipeURI + "'"
        # Aufgrund von \\'-Problematiken wird hier das ganze Angegeben.
        if "Swirled_Peppermint_Marshmallows" in row:
            recipeURI = "http://purl.org/ProductKG/RecipeOn#Liz's_Swirled_Peppermint_Marshmallows."
        # _Recipe für Baked Chicken Breasts Crockpot
        if "Baked_Chicken_Breasts_Crockpot" in row:
            recipeURI = recipeURI+"_Recipe"
        # Bilde schließlich nun die URI
        recipeURI=URIRef(recipeURI)
    # Falls das erste Zeichen eine Eckenklammeröffnung ist, und somit als ein gefiltertes Verb von dem Java-Programm und CoreNLP gilt...
    if (row[0] == '['):
        # Finde die erste Leerzeichen
        laterNumber = row.find(" ")
        #Bestimme das Wort aus der Zeile via :-Splicing
        #Anfang ist Konsistent -> 6 
        #Minus-Multiplikation führt zum Negative Splicing, dies relativiert auf die Gräße der Datei führt schließlich zur Isolation des Verbes aus der Zeile
        #word = row[6:-1*(len(row) - laterNumber)].casefold()
        word = row[6:laterNumber].casefold()
        match word:
            case "cut":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/SOMA.owl#Cutting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))   
            case "cutting":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/SOMA.owl#Cutting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "carve":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Carving")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "carved":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Carving")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "carving":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Carving")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slash":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Slashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slashed":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Slashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slashing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Slashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slice":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/SOMA.owl#Slicing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "sliced":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/SOMA.owl#Slicing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slicing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/SOMA.owl#Slicing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "dice":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/SOMA.owl#Dicing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "diced":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/SOMA.owl#Dicing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "dicing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/SOMA.owl#Dicing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "hash":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Hashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "hashed":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Hashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "hashing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Hashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "mince":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Mincing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "minced":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Mincing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "mincing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Mincing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "quarter":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Quartering")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "quatered":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Quartering")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "quatering":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Quartering")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "saw":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Sawing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "sawed":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Sawing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "sawing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Sawing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slit":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Slitting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slitting":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Slitting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "sliver":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Slivering")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slivered":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Slivering")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "slivering":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Slivering")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "snip":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Snipping")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "snipped":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Snipping")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "snipping":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Snipping")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "halve":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Halving")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "halved":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Halving")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "halving":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Halving")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "cube":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Cubing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "cubed":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Cubing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "cubing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Cubing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "sever":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Severing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "severed":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Severing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "servering":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Severing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "crosscut":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Crosscutting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "crosscutting":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Crosscutting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new)) 
            case "incise":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Incising")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "incise":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Incising")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "incising":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Incising")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "jag":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Jagging")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "jag":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Jagging")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "jagging":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Jagging")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "julienne":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Julienning")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "julienned":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Julienning")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "julienning":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Julienning")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "trench":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Trenching")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "trenched":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Trenching")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "trenching":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Trenching")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "trisect":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Trisecting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "trisected":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Trisecting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "trisecting":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Trisecting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "chop":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Chopping")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "chopped":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Chopping")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "chopping":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Chopping")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "fillet":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Filletting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "filleted":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Filletting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "filleting":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Filletting")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "pare":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Paring")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "pared":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Paring")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "paring":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/food_cutting#Paring")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "pour": 
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Pouring")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "poured":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Pouring")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "pouring":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Pouring")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "cascade":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Cascading")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "cascaded":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Cascading")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "cascading":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Cascading")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "flow":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Flowing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "flowed":
                 # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Flowing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "flowing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Flowing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "drain":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Draining")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "drained":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Draining")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "draining":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Draining")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "sprinkle":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Sprinkling")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "sprinkled":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Sprinkling")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "sprinkling":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Sprinkling")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "splash":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Splashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "splashed":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Splashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "splashing":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Splashing")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "spill":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Spilling")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "spilled":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Spilling")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "spilling":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Spilling")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "stream":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Streaming")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "streamed":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Streaming")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))
            case "streaming":
                # Kreiere den neuen Knoten und addiere ihn zum Graphen als Restriction
                 new = BNode()                
                 graph.add((new,RDF.type,OWL.Restriction))
                # Verbinde das neue zur Object Property includes_task 
                 graph.add((new, OWL.onProperty, includesTask))
                # Referenziere den im Rezept vorhandenen Verb
                 graph.add((new, OWL.someValuesFrom, URIRef("http://www.ease-crc.org/ont/pouring_liquids#Streaming")))
                  # Schließlich addiere die neue Bedingung zum Rezept als Subclass
                 graph.add((recipeURI ,RDFS.subClassOf, new))

#Speichere die in RDFLIB geöffnete Ontologie als RDF/XML
graph.serialize(destination = "/home/jovyan/work/Linking/Intermediary.owl",format="xml")
#Öffne sie wieder mit OWLready2
testOwl = OWLREADY.get_ontology("/home/jovyan/work/Linking/Intermediary.owl").load()
#Speichere sie nun wieder, nun mit OWLReady2 -> Weicht zugriffsproblem der Betreuerin aus.
testOwl.save(file = "/home/jovyan/work/Linking/Finished.owl", format = "rdfxml")

In [43]:
newGraph = Graph()

newGraph.parse("Finished.owl")

affectedRecipe = []

gettingAffectedRecipe = prepareQuery( 
    """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
	PREFIX recipe: <http://purl.org/ProductKG/RecipeOn#>
	PREFIX owl: <http://www.w3.org/2002/07/owl#>
	PREFIX soma: <http://www.ease-crc.org/ont/SOMA.owl#>
	SELECT DISTINCT ?recipe 	
	WHERE{
		?recipe rdfs:subClassOf ?a.
		?a owl:onProperty soma:includes_task.}
    """,
)

for row in newGraph.query(gettingAffectedRecipe):
    affectedRecipe.append(row.recipe)
print(len(affectedRecipe))

6111


In [42]:
newGraph = Graph()

newGraph.parse("Finished.owl")
cutAmount = []
pourAmount = []
getCutAmount = prepareQuery(
    """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
	PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX task: <http://www.ease-crc.org/ont/>
	SELECT ?d
    WHERE{
        ?d owl:someValuesFrom ?e.
        ?e rdfs:subClassOf task:food_cutting
    }
    """,
)

getPourAmount = prepareQuery(
    """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX task: <http://www.ease-crc.org/ont/>
SELECT ?d
WHERE{
    ?d owl:someValuesFrom ?e.
    ?e rdfs:subClassOf task:pouring_liquids
}
""",
)
for row in newGraph.query(getCutAmount):
    cutAmount.append(row.d)
print(len(cutAmount))
for row in newGraph.query(getPourAmount):
    pourAmount.append(row.d)
print(len(pourAmount))
print(len(cutAmount)+len(pourAmount))

5906
6943
12849


In [12]:
#print(len(array)+len(array2))

11425


In [52]:
amount = []
getVerb = prepareQuery(
    """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT ?d
WHERE{
    ?d owl:someValuesFrom <http://www.ease-crc.org/ont/pouring_liquids#Streaming>.
    }
    """,
)
for row in newGraph.query(getVerb):
    amount.append(row.d)
print(len(amount))

0
